# Adding constraints derived from a structure

In [ ]:
# You have to rerun this cell each time you start a new notebook or do a "factory reset".
import sys
if 'google.colab' in sys.modules:
    !pip install pyrosettacolabsetup
    import pyrosettacolabsetup
    pyrosettacolabsetup.mount_pyrosetta_install()
    print ("Notebook is set for PyRosetta use in Colab.  Have fun!")

Now cd into the correct directory

In [ ]:
# cd into the correct directory

We initialize PyRosetta without any command line options.

In [ ]:
from pyrosetta import *
init()

Initiliazing pose and score function.

In [ ]:
pose = pose_from_pdb("inputs/1ubq.pdb")
sfxn = get_fa_scorefxn()
sfxn(pose)

Let's prime the score function with a constraint score term. Let's also print out the score before we've added any constraints to the pose so we can refer back to it later.

In [ ]:
from pyrosetta.rosetta.core.scoring import *
sfxn.set_weight(atom_pair_constraint, 1.0)
sfxn.show(pose)

To define where the constraints need to be applied, we will need to use residue selectors. Let's use a `LayerSelector` to select the surface residues.

In [ ]:
from pyrosetta.rosetta.core.select.residue_selector import *
surface_res = LayerSelector()
surface_res.set_layers(False, False, True) # use "?" to look at this function arguments

To check which residues are selected on the pose, we can apply it. `1` means that the residue has been selected (and is a surface residue), `0` means that it has not.

In [ ]:
surface_res.apply(pose)

Now let's create atom pair constraints. For this, we use `AtomPairConstraintGenerator`. **By default, it uses the Sum Of Gaussians function to score the distances between atoms.** The "sum" part is not really used, because there is just one gaussian. This gaussian turns a distance into a probability, `prob`, and the energy is taken as the `-ln(prob)`.   We will pass the residue selector to the generator, set the maximum distance at under which it will create atom pair constraints (5.0 Ang), the standard deviation of the gaussians, and set the constraints to only be generated between pairs of C-alpha atoms.

In [ ]:
from pyrosetta.rosetta.protocols.constraint_generator import *
apcg = AtomPairConstraintGenerator()
apcg.set_residue_selector(surface_res)
apcg.set_max_distance(5.0)
apcg.set_sd(1.0)
apcg.set_ca_only(True)

While we could directly apply the `AtomPairConstraintGenerator` to the pose, the recommended method is to add it to an `AddConstraints` object. Multiple constraint generators can be added to this. We then apply all the constraints.

In [ ]:
add_csts = AddConstraints()
add_csts.add_generator(apcg)
add_csts.apply(pose)

Let's check if the constraints were applied.

In [ ]:
sfxn.show(pose)

The sum of gaussians function acts more like a reward. A large neagtive number indicates that all 9 constraints were satisfied. What happens when we disrupt some surface interactions in a perturbed structure?

In [ ]:
pose_perturbed = pose.clone()  # copy the conformation and the constraints
pose_perturbed.set_phi(25, -130)
pose_perturbed.set_psi(25,  145)

In [ ]:
sfxn.show(pose_perturbed)

The reward is much smaller owing to the perturbation of the pose.